# 🐍 Лекция 4: Инкапсуляция по-питоньи

<div style="background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); padding: 20px; border-radius: 10px; color: white;">
  <h2 style="color: white;">Как Python защищает данные? Или не защищает?</h2>
  <p>Сегодня мы разберёмся с одной из ключевых концепций ООП — инкапсуляцией. Узнаем, чем подход Python отличается от C++ и Java.</p>
</div>

## 🎯 Цели лекции

✅ Понять, что такое инкапсуляция и зачем она нужна  
✅ Изучить соглашения Python: `_protected` и `__private`  
✅ Освоить декоратор `@property` для управления доступом  
✅ Научиться использовать `@classmethod` и `@staticmethod`  
✅ Понять разницу между методами экземпляра, класса и статическими  

## 🔐 Что такое инкапсуляция?

**Инкапсуляция** — это механизм сокрытия внутренней реализации объекта и предоставления контролируемого доступа к его данным.

### Зачем нужна инкапсуляция?
1. **Защита данных** — предотвращение некорректного изменения
2. **Гибкость** — возможность изменить реализацию без изменения интерфейса
3. **Упрощение использования** — пользователь видит только нужные методы
4. **Контроль доступа** — кто и как может изменять данные

### Как в C++/Java vs Как в Python:
- **C++/Java**: `private`, `protected`, `public` — компилятор запрещает доступ
- **Python**: `_`, `__` — соглашения, а не запреты (мы "взрослые люди")

## 📊 План лекции

1. **Соглашения об именовании** — `public`, `_protected`, `__private`
2. **Декоратор `@property`** — геттеры и сеттеры по-питоньи
3. **Методы класса `@classmethod`** — работа на уровне класса
4. **Статические методы `@staticmethod`** — утилитарные функции в классе
5. **Практика** — когда что использовать

> 💡 **Ключевая идея:** В Python доверяют программисту. Соглашения важнее запретов.

## 🔍 Уровни доступа в Python

### 1. Public (публичные) — доступны всем
- Имена без подчёркиваний: `name`, `calculate()`
- Могут использоваться где угодно
- "Интерфейс" класса — что видят пользователи

### 2. Protected (защищённые) — для внутреннего использования
- Одно подчёркивание в начале: `_name`, `_calculate()`
- **Соглашение**: "не используйте снаружи, если не знаете, что делаете"
- Не запрещено, но не рекомендуется
- Видны при наследовании

### 3. Private (приватные) — для сокрытия реализации
- Два подчёркивания в начале: `__name`, `__calculate()`
- **Name mangling** — Python меняет имя, чтобы избежать случайных конфликтов
- Технически доступны, но сложнее
- Используются реже, чем `_`

### Важное отличие от C++/Java:
В Python нет настоящей защиты на уровне языка! Это соглашения, которые:
1. **Предупреждают** разработчика
2. **Помогают избежать** случайных ошибок
3. **Не мешают** осознанному доступу при необходимости

In [ ]:
# Демонстрация уровней доступа
print("🔍 Уровни доступа в Python:\n")

class BankAccount:
    """Класс банковского счёта с разными уровнями доступа."""
    
    # Public атрибут класса (используется редко)
    bank_name = "Python Bank"
    
    def __init__(self, owner, balance=0):
        # Public атрибуты (видимы всем)
        self.owner = owner
        
        # Protected атрибут (одно подчёркивание)
        # "Внутренняя" переменная, лучше не трогать снаружи
        self._balance = balance
        
        # Private атрибут (два подчёркивания)
        # Python изменит имя, чтобы избежать конфликтов
        self.__pin = 1234
        
        # Protected метод (для внутреннего использования)
        self._log_creation()
    
    def _log_creation(self):
        """Protected метод - для внутреннего использования класса."""
        print(f"[LOG] Создан счёт для {self.owner}")
    
    def __validate_pin(self, pin):
        """Private метод - не предназначен для вызова извне."""
        return self.__pin == pin
    
    # Public методы (интерфейс класса)
    def deposit(self, amount):
        """Пополнить счёт."""
        if amount > 0:
            self._balance += amount
            return True
        return False
    
    def get_balance(self, pin):
        """Получить баланс (требует пин)."""
        if self.__validate_pin(pin):
            return self._balance
        return "Неверный пин-код"

# Создаём объект
print("Создаём банковский счёт:")
account = BankAccount("Иван Иванов", 1000)
print()

# 1. Доступ к public атрибутам и методам
print("1. Public доступ (всё разрешено):")
print(f"   account.owner = {account.owner}")
print(f"   account.deposit(500) = {account.deposit(500)}")
print(f"   account.get_balance(1234) = {account.get_balance(1234)}")
print()

# 2. Доступ к protected атрибутам и методам (не рекомендуется, но можно)
print("2. Protected доступ (можно, но не нужно):")
print(f"   account._balance = {account._balance}")
print(f"   account._log_creation() работает:")
account._log_creation()
print()

# 3. Попытка доступа к private атрибутам
print("3. Private доступ (Python меняет имена):")
print(f"   account.__pin -> Ошибка! Атрибут не найден")
try:
    print(f"   account.__pin = {account.__pin}")
except AttributeError as e:
    print(f"   Ошибка: {e}")

# Но можно посмотреть, как Python изменил имя
print(f"\n   Смотрим dir(account):")
private_attrs = [attr for attr in dir(account) if "pin" in attr]
print(f"   Атрибуты с 'pin': {private_attrs}")
print(f"   Настоящее имя: {private_attrs[0]}")

# И даже получить доступ (но НЕ ДЕЛАЙТЕ ТАК без веской причины!)
print(f"\n   Но можно получить доступ через изменённое имя:")
real_name = private_attrs[0]
print(f"   account.{real_name} = {getattr(account, real_name)}")

# 4. Доступ к private методам
print(f"\n4. Private методы:")
try:
    account.__validate_pin(1234)
except AttributeError as e:
    print(f"   account.__validate_pin(1234) -> Ошибка: {e}")

# Но тоже можно через изменённое имя
print(f"\n   Находим private метод:")
private_methods = [attr for attr in dir(account) if "validate" in attr]
print(f"   Методы с 'validate': {private_methods}")

if private_methods:
    method_name = private_methods[0]
    method = getattr(account, method_name)
    print(f"   account.{method_name}(1234) = {method(1234)}")
    print(f"   account.{method_name}(9999) = {method(9999)}")

# 5. Что такое name mangling?
print("\n5. Name Mangling (искажение имён):")
print("   Когда Python видит __имя, он меняет его на _ИмяКласса__имя")
print(f"   В нашем случае: __pin -> _BankAccount__pin")
print(f"   Это помогает избежать конфликтов при наследовании")

In [ ]:
# Демонстрация Name Mangling
print("🔧 Name Mangling в действии:\n")

class Parent:
    def __init__(self):
        self.public = "public"
        self._protected = "protected"
        self.__private = "private"
    
    def show_names(self):
        print(f"Изнутри класса Parent:")
        print(f"  self.public = {self.public}")
        print(f"  self._protected = {self._protected}")
        print(f"  self.__private = {self.__private}")
        
        # Посмотрим реальные имена
        print(f"\n  Реальные имена в self.__dict__:")
        for key in self.__dict__:
            print(f"    {key}: {self.__dict__[key]}")

class Child(Parent):
    def __init__(self):
        super().__init__()
        # Пытаемся переопределить атрибуты
        self.__private = "child private"  # Это ДРУГОЙ атрибут!
    
    def show_child_names(self):
        print(f"\nИзнутри класса Child:")
        print(f"  self.public = {self.public}")
        print(f"  self._protected = {self._protected}")
        
        # Пытаемся получить доступ к __private родителя
        try:
            print(f"  self.__private (родитель) = {self.__private}")
        except AttributeError:
            print(f"  self.__private = AttributeError (не найден)")
        
        # Но у нас есть свой __private
        print(f"  У Child есть свой __private: {self.__private}")
        
        print(f"\n  Все атрибуты в self.__dict__:")
        for key in self.__dict__:
            print(f"    {key}: {self.__dict__[key]}")

# Создаём объекты
print("Создаём объект Parent:")
parent = Parent()
parent.show_names()

print("\n" + "="*50 + "\n")

print("Создаём объект Child:")
child = Child()
child.show_names()
print()
child.show_child_names()

# Доступ извне
print("\n" + "="*50 + "\n")
print("Доступ извне к объектам:")

print("\nДля parent:")
print(f"  parent.public = {parent.public}")
print(f"  parent._protected = {parent._protected}")

# Доступ к private через mangled имя
print(f"  parent._Parent__private = {parent._Parent__private}")

print("\nДля child:")
print(f"  child.public = {child.public}")
print(f"  child._protected = {child._protected}")
print(f"  child._Parent__private = {child._Parent__private}")
print(f"  child._Child__private = {child._Child__private}")

# Визуализация name mangling
print("\n" + "="*50 + "\n")
print("🎨 Визуализация Name Mangling:")

print("""
Parent:                          Child:
  public                         public
  _protected                     _protected
  __private  → _Parent__private  __private  → _Child__private
                                 _Parent__private (унаследован, но скрыт)
                                 
Результат: у Child ДВА разных "private" атрибута!
1. _Parent__private - от родителя
2. _Child__private - свой собственный
""")

## 🎯 Когда что использовать?

### Public (`name`):
- **Когда:** Методы и атрибуты, которые являются частью публичного API
- **Примеры:**
  - `save()`, `load()`, `calculate()` — основные операции
  - `name`, `id` — основные данные объекта
- **Правило:** Всё, что нужно пользователю класса

### Protected (`_name`):
- **Когда:** Для внутренней реализации, но может понадобиться при наследовании
- **Примеры:**
  - `_cache`, `_configuration` — внутренние данные
  - `_validate_input()`, `_prepare_data()` — вспомогательные методы
- **Правило:** "Не трогай, если не знаешь, что делаешь"

### Private (`__name`):
- **Когда:** Для полного сокрытия реализации, особенно при наследовании
- **Примеры:**
  - `__secret_key`, `__algorithm` — действительно секретные данные
  - `__implementation_details()` — методы, которые не должны переопределяться
- **Правило:** Используйте редко, в основном для избежания конфликтов имен

### Классическая ошибка новичков:
```python
# ❌ Плохо: Создавать private всё подряд
class BadExample:
    def __init__(self):
        self.__name = ""  # Зачем? Используйте _name
        self.__age = 0    # Или просто age, если это публичный атрибут
        self.__calculate() # Избыточная приватность

# ✅ Хорошо: Осознанное использование
class GoodExample:
    def __init__(self, name, age):
        self.name = name      # Public - часть API
        self._age = age       # Protected - может понадобиться наследникам
        self.__secret = 123   # Private - действительно скрываем


---

## Декоратор `@property` — теория (Markdown)


## 🎭 Декоратор `@property` — геттеры и сеттеры по-питоньи

### Проблема: как контролировать доступ к атрибутам?
В C++/Java используют геттеры и сеттеры:

// Java-стиль
public class Person {
    private String name;
    
    public String getName() { return name; }
    public void setName(String name) { 
        if (name != null) this.name = name; 
    }
}

In [ ]:
# Демонстрация @property
print("🎭 Декоратор @property в действии:\n")

# Пример 1: Простой property с валидацией
print("1. Person с property для возраста:")

class Person:
    def __init__(self, name, age):
        self.name = name
        self._age = None  # Храним в protected атрибуте
        self.age = age  # Используем setter для валидации
    
    @property
    def age(self):
        """Геттер для возраста."""
        print(f"   [GET] Возраст {self.name}: {self._age}")
        return self._age
    
    @age.setter
    def age(self, value):
        """Сеттер для возраста с валидацией."""
        if not isinstance(value, int):
            raise TypeError("Возраст должен быть целым числом")
        if value < 0 or value > 150:
            raise ValueError("Возраст должен быть от 0 до 150")
        
        old_age = self._age
        self._age = value
        print(f"   [SET] {self.name}: {old_age} → {value}")
    
    @age.deleter
    def age(self):
        """Делитер для возраста."""
        print(f"   [DEL] Удаляем возраст {self.name}")
        self._age = None
    
    @property
    def is_adult(self):
        """Вычисляемое свойство (только геттер)."""
        return self._age >= 18 if self._age is not None else False

# Используем
print("Создаём объект:")
person = Person("Иван", 25)
print()

print("Работа с property:")
print(f"  person.age = {person.age}")  # Вызывает геттер
print(f"  person.is_adult = {person.is_adult}")  # Вычисляемое свойство
print()

print("Меняем возраст:")
person.age = 30  # Вызывает сеттер с валидацией
print(f"  Новый возраст: {person.age}")
print()

print("Пытаемся установить невалидный возраст:")
try:
    person.age = -5
except ValueError as e:
    print(f"  Ошибка: {e}")
print()

print("Удаляем возраст:")
del person.age
print(f"  Возраст после удаления: {person.age}")
print()

# Пример 2: Property для вычисляемых значений
print("2. Rectangle с вычисляемыми свойствами:")

class Rectangle:
    def __init__(self, width, height):
        self._width = width
        self._height = height
    
    @property
    def width(self):
        return self._width
    
    @width.setter
    def width(self, value):
        if value <= 0:
            raise ValueError("Ширина должна быть положительной")
        self._width = value
    
    @property
    def height(self):
        return self._height
    
    @height.setter
    def height(self, value):
        if value <= 0:
            raise ValueError("Высота должна быть положительной")
        self._height = value
    
    @property
    def area(self):
        """Вычисляемая площадь (только чтение)."""
        return self._width * self._height
    
    @property
    def perimeter(self):
        """Вычисляемый периметр (только чтение)."""
        return 2 * (self._width + self._height)
    
    @property
    def is_square(self):
        """Вычисляемое свойство: является ли квадратом."""
        return self._width == self._height

# Используем
rect = Rectangle(5, 10)
print(f"  Прямоугольник {rect.width}x{rect.height}")
print(f"  Площадь: {rect.area}")
print(f"  Периметр: {rect.perimeter}")
print(f"  Квадрат? {rect.is_square}")
print()

# Меняем размеры
rect.width = 10
print(f"  После rect.width = 10:")
print(f"  Площадь: {rect.area}")
print(f"  Квадрат? {rect.is_square}")
print()

# Пример 3: Property для обратной совместимости
print("3. Property для обратной совместимости:")

class OldClass:
    """Старая версия класса с публичным атрибутом."""
    def __init__(self, value):
        self.value = value  # Публичный атрибут

# Допустим, мы использовали этот класс
old = OldClass(10)
print(f"  Старая версия: old.value = {old.value}")

# Новая версия с property
class NewClass:
    """Новая версия с property для контроля доступа."""
    def __init__(self, value):
        self._value = value
    
    @property
    def value(self):
        return self._value
    
    @value.setter
    def value(self, new_value):
        # Добавили валидацию
        if new_value < 0:
            raise ValueError("Значение не может быть отрицательным")
        self._value = new_value

# Код, использующий класс, НЕ ИЗМЕНИЛСЯ!
new = NewClass(10)
print(f"  Новая версия: new.value = {new.value}")
new.value = 20
print(f"  После new.value = 20: {new.value}")

try:
    new.value = -5
except ValueError as e:
    print(f"  Ошибка при new.value = -5: {e}")

print("\n✅ Вывод: @property позволяет менять реализацию без изменения API!")

## 👥 Методы класса: `@classmethod`

### Что такое методы класса?
Методы, которые работают с классом в целом, а не с конкретным объектом.

### Особенности:
1. **Первый параметр** — `cls` (ссылка на класс, не на объект)
2. **Вызываются от класса** — `Класс.метод()` или `объект.метод()`
3. **Имеют доступ к атрибутам класса** — `cls.атрибут`
4. **Не имеют доступа к атрибутам экземпляра** — нет `self`

### Когда использовать `@classmethod`?
1. **Альтернативные конструкторы** — создание объектов разными способами
2. **Фабричные методы** — создание объектов сложным образом
3. **Работа с атрибутами класса** — изменение настроек для всех объектов
4. **Утилиты, связанные с классом** — операции, не требующие экземпляра

### Синтаксис:
```python
class MyClass:
    @classmethod
    def method(cls, параметры):
        # cls - ссылка на класс
        # Можно использовать cls.атрибуты_класса
        pass

In [ ]:
# Демонстрация @classmethod
print("👥 Методы класса (@classmethod) в действии:\n")

# Пример 1: Альтернативные конструкторы
print("1. Date с альтернативными конструкторами:")

class Date:
    """Класс для работы с датами."""
    
    def __init__(self, day, month, year):
        self.day = day
        self.month = month
        self.year = year
    
    def __str__(self):
        return f"{self.day:02d}.{self.month:02d}.{self.year}"
    
    @classmethod
    def from_string(cls, date_string):
        """
        Альтернативный конструктор из строки 'dd.mm.yyyy'.
        
        Args:
            date_string: Дата в формате 'dd.mm.yyyy'
        
        Returns:
            Объект Date
        """
        day, month, year = map(int, date_string.split('.'))
        return cls(day, month, year)  # cls() вместо Date()
    
    @classmethod
    def from_american_format(cls, date_string):
        """
        Альтернативный конструктор из американского формата 'mm/dd/yyyy'.
        """
        month, day, year = map(int, date_string.split('/'))
        return cls(day, month, year)
    
    @classmethod
    def today(cls):
        """
        Создаёт объект Date с текущей датой.
        Пример фабричного метода.
        """
        import datetime
        now = datetime.datetime.now()
        return cls(now.day, now.month, now.year)

# Используем разные конструкторы
print("Создаём даты разными способами:")
date1 = Date(15, 1, 2024)  # Обычный конструктор
print(f"  Date(15, 1, 2024) = {date1}")

date2 = Date.from_string("20.01.2024")  # Альтернативный конструктор
print(f"  Date.from_string('20.01.2024') = {date2}")

date3 = Date.from_american_format("01/25/2024")  # Ещё один конструктор
print(f"  Date.from_american_format('01/25/2024') = {date3}")

# date4 = Date.today()  # Раскомментируйте, чтобы увидеть текущую дату
# print(f"  Date.today() = {date4}")
print()

# Пример 2: Работа с атрибутами класса
print("2. Configuration с classmethod для управления настройками:")

class Configuration:
    """Класс для управления настройками приложения."""
    
    # Атрибуты класса (общие настройки)
    default_language = "en"
    debug_mode = False
    version = "1.0.0"
    
    def __init__(self, user):
        self.user = user
        self.user_language = Configuration.default_language
    
    @classmethod
    def set_default_language(cls, language):
        """Устанавливает язык по умолчанию для ВСЕХ конфигураций."""
        cls.default_language = language
        print(f"[CONFIG] Язык по умолчанию изменён на: {language}")
    
    @classmethod
    def enable_debug(cls):
        """Включает режим отладки для ВСЕХ конфигураций."""
        cls.debug_mode = True
        print("[CONFIG] Режим отладки включён")
    
    @classmethod
    def get_config_info(cls):
        """Возвращает информацию о конфигурации."""
        return {
            "version": cls.version,
            "default_language": cls.default_language,
            "debug_mode": cls.debug_mode
        }

# Создаём конфигурации
print("Создаём конфигурации:")
config1 = Configuration("user1")
config2 = Configuration("user2")

print(f"  config1.user_language = {config1.user_language}")
print(f"  config2.user_language = {config2.user_language}")
print(f"  Configuration.default_language = {Configuration.default_language}")
print()

# Меняем настройки на уровне класса
print("Меняем настройки через classmethod:")
Configuration.set_default_language("ru")

print(f"\nПосле изменения:")
print(f"  config1.user_language = {config1.user_language}")
print(f"  config2.user_language = {config2.user_language}")
print(f"  Configuration.default_language = {Configuration.default_language}")

# Но config1 уже создан с языком 'en'!
print(f"\n⚠️ Внимание: config1 был создан ДО изменения языка!")
print(f"  config1.user_language всё ещё 'en' (установлен в __init__)")
print(f"  config2.user_language тоже 'en' (тоже установлен в __init__)")
print(f"  Но новые объекты будут создаваться с языком 'ru'")

# Создаём новый объект после изменения
config3 = Configuration("user3")
print(f"\nНовый объект после изменения:")
print(f"  config3.user_language = {config3.user_language}")
print()

# Пример 3: Фабричные методы
print("3. Product с фабричными методами:")

class Product:
    """Класс продукта с фабричными методами."""
    
    def __init__(self, name, price, category):
        self.name = name
        self.price = price
        self.category = category
    
    def __str__(self):
        return f"{self.name} ({self.category}): ${self.price}"
    
    @classmethod
    def create_book(cls, title, author, price):
        """Фабричный метод для создания книг."""
        return cls(
            name=f"{title} - {author}",
            price=price,
            category="book"
        )
    
    @classmethod
    def create_electronics(cls, name, brand, price):
        """Фабричный метод для создания электроники."""
        return cls(
            name=f"{brand} {name}",
            price=price,
            category="electronics"
        )
    
    @classmethod
    def create_from_dict(cls, data):
        """Создание продукта из словаря."""
        return cls(
            name=data.get("name", "Unknown"),
            price=data.get("price", 0),
            category=data.get("category", "other")
        )

# Используем фабричные методы
print("Создаём продукты фабричными методами:")
book = Product.create_book("Преступление и наказание", "Достоевский", 20)
print(f"  Product.create_book(...) = {book}")

laptop = Product.create_electronics("Ноутбук", "Dell", 1000)
print(f"  Product.create_electronics(...) = {laptop}")

data = {"name": "Молоко", "price": 2, "category": "food"}
milk = Product.create_from_dict(data)
print(f"  Product.create_from_dict({{...}}) = {milk}")

print("\n✅ Вывод: @classmethod отлично подходит для альтернативных конструкторов и фабрик!")

## 🛠️ Статические методы: `@staticmethod`

### Что такое статические методы?
Методы, которые не имеют доступа ни к экземпляру (`self`), ни к классу (`cls`).

### Особенности:
1. **Нет параметра `self` или `cls`** — просто обычная функция внутри класса
2. **Не имеют доступа к атрибутам** — ни экземпляра, ни класса
3. **Вызываются от класса или объекта** — `Класс.метод()` или `объект.метод()`
4. **Логически связаны с классом** — но не требуют его состояния

### Когда использовать `@staticmethod`?
1. **Утилитарные функции** — вспомогательные методы, связанные с классом по смыслу
2. **Функции-помощники** — которые могут быть полезны при работе с классом
3. **Альтернатива модульным функциям** — когда логичнее разместить в классе
4. **Методы, которые не изменяют состояние** — чистые функции

### Синтаксис:
```python
class MyClass:
    @staticmethod
    def method(параметры):
        # Нет self, нет cls
        # Просто функция
        pass

In [ ]:
# Демонстрация @staticmethod
print("🛠️ Статические методы (@staticmethod) в действии:\n")

# Пример 1: Утилитарные функции в классе
print("1. MathUtils со статическими методами:")

class MathUtils:
    """Класс с математическими утилитами."""
    
    @staticmethod
    def add(a, b):
        """Сложение двух чисел."""
        return a + b
    
    @staticmethod
    def multiply(a, b):
        """Умножение двух чисел."""
        return a * b
    
    @staticmethod
    def factorial(n):
        """Вычисление факториала."""
        if n < 0:
            raise ValueError("Факториал определён только для неотрицательных чисел")
        result = 1
        for i in range(2, n + 1):
            result *= i
        return result
    
    @staticmethod
    def is_prime(n):
        """Проверка, является ли число простым."""
        if n < 2:
            return False
        for i in range(2, int(n**0.5) + 1):
            if n % i == 0:
                return False
        return True

# Используем статические методы
print("Используем статические методы:")
print(f"  MathUtils.add(5, 3) = {MathUtils.add(5, 3)}")
print(f"  MathUtils.multiply(4, 7) = {MathUtils.multiply(4, 7)}")
print(f"  MathUtils.factorial(5) = {MathUtils.factorial(5)}")
print(f"  MathUtils.is_prime(17) = {MathUtils.is_prime(17)}")
print(f"  MathUtils.is_prime(18) = {MathUtils.is_prime(18)}")

# Можно вызывать и от экземпляра (но зачем?)
utils = MathUtils()
print(f"\nОт экземпляра: utils.add(2, 3) = {utils.add(2, 3)}")
print()

# Пример 2: Валидация данных
print("2. Validator со статическими методами проверки:")

class Validator:
    """Класс для валидации данных."""
    
    @staticmethod
    def validate_email(email):
        """Проверяет валидность email."""
        if not email or "@" not in email:
            return False
        # Простая проверка
        return len(email) > 5
    
    @staticmethod
    def validate_phone(phone):
        """Проверяет валидность телефона."""
        if not phone:
            return False
        # Удаляем все нецифровые символы
        digits = ''.join(filter(str.isdigit, phone))
        return 10 <= len(digits) <= 15
    
    @staticmethod
    def validate_password(password):
        """Проверяет сложность пароля."""
        if not password or len(password) < 8:
            return False
        
        has_upper = any(c.isupper() for c in password)
        has_lower = any(c.islower() for c in password)
        has_digit = any(c.isdigit() for c in password)
        
        return has_upper and has_lower and has_digit
    
    @staticmethod
    def validate_russian_inn(inn):
        """Проверяет валидность ИНН (Россия)."""
        if not inn or not inn.isdigit():
            return False
        
        if len(inn) not in (10, 12):
            return False
        
        # Здесь была бы настоящая проверка контрольных сумм
        # Для примера упростим
        return True

# Используем
print("Валидация данных:")
print(f"  Validator.validate_email('test@example.com') = {Validator.validate_email('test@example.com')}")
print(f"  Validator.validate_email('invalid') = {Validator.validate_email('invalid')}")

print(f"  Validator.validate_phone('+7 (999) 123-45-67') = {Validator.validate_phone('+7 (999) 123-45-67')}")
print(f"  Validator.validate_phone('123') = {Validator.validate_phone('123')}")

print(f"  Validator.validate_password('StrongPass123') = {Validator.validate_password('StrongPass123')}")
print(f"  Validator.validate_password('weak') = {Validator.validate_password('weak')}")
print()

# Пример 3: Форматирование и преобразование
print("3. Formatter со статическими методами:")

class Formatter:
    """Класс для форматирования данных."""
    
    @staticmethod
    def format_price(amount, currency="USD"):
        """Форматирует цену."""
        if currency == "USD":
            return f"${amount:.2f}"
        elif currency == "EUR":
            return f"€{amount:.2f}"
        elif currency == "RUB":
            return f"{amount:.2f} ₽"
        else:
            return f"{amount:.2f} {currency}"
    
    @staticmethod
    def format_date(date_string, input_format="%Y-%m-%d", output_format="%d.%m.%Y"):
        """Форматирует дату из одного формата в другой."""
        from datetime import datetime
        try:
            date = datetime.strptime(date_string, input_format)
            return date.strftime(output_format)
        except ValueError:
            return date_string
    
    @staticmethod
    def truncate_text(text, max_length=100, suffix="..."):
        """Обрезает текст до максимальной длины."""
        if len(text) <= max_length:
            return text
        return text[:max_length - len(suffix)] + suffix
    
    @staticmethod
    def bytes_to_human(size_bytes):
        """Преобразует байты в человекочитаемый формат."""
        for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
            if size_bytes < 1024.0:
                return f"{size_bytes:.2f} {unit}"
            size_bytes /= 1024.0
        return f"{size_bytes:.2f} PB"

# Используем
print("Форматирование данных:")
print(f"  Formatter.format_price(1234.567) = {Formatter.format_price(1234.567)}")
print(f"  Formatter.format_price(1234.567, 'RUB') = {Formatter.format_price(1234.567, 'RUB')}")

print(f"  Formatter.format_date('2024-01-25') = {Formatter.format_date('2024-01-25')}")
print(f"  Formatter.format_date('25.01.2024', '%d.%m.%Y', '%Y-%m-%d') = {Formatter.format_date('25.01.2024', '%d.%m.%Y', '%Y-%m-%d')}")

text = "Очень длинный текст, который нужно обрезать для отображения в интерфейсе"
print(f"  Formatter.truncate_text('{text[:20]}...') = {Formatter.truncate_text(text, 30)}")

print(f"  Formatter.bytes_to_human(1024) = {Formatter.bytes_to_human(1024)}")
print(f"  Formatter.bytes_to_human(1500000) = {Formatter.bytes_to_human(1500000)}")

print("\n✅ Вывод: @staticmethod отлично подходит для утилитарных функций, связанных с классом!")

In [ ]:
# Сравнение всех типов методов
print("📊 Сравнение всех типов методов в Python:\n")

class ExampleClass:
    """Класс для демонстрации всех типов методов."""
    
    # Атрибут класса
    class_attribute = "Я атрибут класса"
    
    def __init__(self, value):
        # Атрибут экземпляра
        self.instance_attribute = value
    
    # 1. Метод экземпляра (обычный метод)
    def instance_method(self):
        """Имеет доступ ко всему."""
        print(f"  1. instance_method:")
        print(f"     self.instance_attribute = {self.instance_attribute}")
        print(f"     self.class_attribute = {self.class_attribute}")
        print(f"     Может вызывать: self.class_method() и self.static_method()")
        return "Результат instance_method"
    
    # 2. Метод класса
    @classmethod
    def class_method(cls):
        """Имеет доступ только к классу."""
        print(f"  2. class_method:")
        print(f"     cls.class_attribute = {cls.class_attribute}")
        print(f"     cls.__name__ = {cls.__name__}")
        print(f"     НЕТ доступа к instance_attribute!")
        print(f"     Может вызывать: cls.static_method()")
        return "Результат class_method"
    
    # 3. Статический метод
    @staticmethod
    def static_method():
        """Не имеет доступа ни к чему."""
        print(f"  3. static_method:")
        print(f"     НЕТ доступа к instance_attribute!")
        print(f"     НЕТ доступа к class_attribute!")
        print(f"     Просто функция в пространстве имён класса")
        return "Результат static_method"

# Создаём объект
print("Создаём объект ExampleClass:")
obj = ExampleClass("Я атрибут экземпляра")
print()

# Вызываем методы разными способами
print("Вызываем методы экземпляра:")
print(f"  obj.instance_method() = {obj.instance_method()}")
print()

print("Вызываем методы класса:")
print(f"  1. От класса: ExampleClass.class_method() = {ExampleClass.class_method()}")
print(f"  2. От экземпляра: obj.class_method() = {obj.class_method()}")
print()

print("Вызываем статические методы:")
print(f"  1. От класса: ExampleClass.static_method() = {ExampleClass.static_method()}")
print(f"  2. От экземпляра: obj.static_method() = {obj.static_method()}")
print()

# Сводная таблица
print("="*60)
print("📋 СВОДНАЯ ТАБЛИЦА: КОГДА ЧТО ИСПОЛЬЗОВАТЬ")
print("="*60)

table = """
| Тип метода      | Декоратор      | 1-й параметр | Доступ к self | Доступ к cls | Когда использовать                  |
|-----------------|----------------|--------------|---------------|--------------|-------------------------------------|
| Метод экземпляра| (нет)          | self         | ДА            | ДА (через self.__class__) | Основная логика работы с объектом  |
| Метод класса    | @classmethod   | cls          | НЕТ           | ДА           | Альтернативные конструкторы, фабрики|
| Статический     | @staticmethod  | (нет)        | НЕТ           | НЕТ          | Утилитарные функции                 |
"""

print(table)
print("="*60)

# Практические примеры использования
print("\n🎯 ПРАКТИЧЕСКИЕ ПРИМЕРЫ ИСПОЛЬЗОВАНИЯ:\n")

print("1. Метод экземпляра — работа с конкретным объектом:")
print("   user.save()          # Сохраняет КОНКРЕТНОГО пользователя")
print("   account.deposit(100) # Пополняет КОНКРЕТНЫЙ счёт")
print()

print("2. Метод класса — работа на уровне класса:")
print("   User.create_from_json(data)  # Создаёт пользователя из JSON")
print("   Config.set_defaults(settings) # Устанавливает настройки для ВСЕХ")
print("   Product.get_all()            # Получает ВСЕ продукты")
print()

print("3. Статический метод — утилитарные функции:")
print("   MathUtils.calculate_distance(x1, y1, x2, y2)")
print("   StringFormatter.truncate(text, length)")
print("   Validator.validate_email(email)")
print()

print("⚠️ ЧАСТЫЕ ОШИБКИ НОВИЧКОВ:\n")

print("1. Использование @staticmethod когда нужен @classmethod:")
print("   ❌ @staticmethod")
print("   def create_user(name):")
print("       return User(name)  # Ошибка! Нужно User(name), но лучше @classmethod")
print()

print("2. Использование @classmethod когда достаточно @staticmethod:")
print("   ❌ @classmethod")
print("   def add(a, b):  # Не использует cls!")
print("       return a + b")
print()

print("3. Путаница с параметрами:")
print("   ❌ @classmethod")
print("   def method(self):  # Должно быть cls, а не self!")
print("       pass")

## 📚 Итоги лекции

### Ключевые концепции:
1. **Инкапсуляция в Python** — это соглашения, а не запреты
2. **`_protected`** — "не трогай, если не знаешь, что делаешь"
3. **`__private`** — name mangling для избежания конфликтов при наследовании
4. **`@property`** — управление доступом к атрибутам с валидацией
5. **`@classmethod`** — методы, работающие с классом (альтернативные конструкторы)
6. **`@staticmethod`** — утилитарные функции в пространстве имён класса

### 🎯 Практические советы:

#### 1. Про инкапсуляцию:
```python
# Используйте public по умолчанию
class User:
    def __init__(self, name):
        self.name = name  # OK
    
    # Используйте _ для внутренних атрибутов
    def _generate_token(self):
        return "token"
    
    # Используйте __ только если действительно нужно
    def __hash_password(self, password):
        return hash(password)

In [ ]:
# Используйте property для валидации
class Temperature:
    def __init__(self, celsius):
        self._celsius = celsius
    
    @property
    def celsius(self):
        return self._celsius
    
    @celsius.setter
    def celsius(self, value):
        if value < -273.15:
            raise ValueError("Температура не может быть ниже абсолютного нуля")
        self._celsius = value
    
    @property
    def fahrenheit(self):
        return self._celsius * 9/5 + 32

In [ ]:
class Database:
    # classmethod - когда нужен доступ к классу
    @classmethod
    def from_config(cls, config_file):
        config = cls._load_config(config_file)
        return cls(**config)
    
    # staticmethod - когда не нужен доступ ни к чему
    @staticmethod
    def validate_connection_string(connection_string):
        return "://" in connection_string